### Increasing the Maize Leaf Disease Dataset Using Contrast Technique

In [ ]:
import PIL
import numpy as np
import os
import tensorflow as tf
import cv2
import keras
import pathlib
import glob
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
import numpy

### Loading of the data

In [ ]:
data_dir=pathlib.Path(os.getcwd()+"/maize_leaf_diseases_dataset/") 
maize_disease_dataset=list(glob.glob(f"{data_dir}/data/*.jpg"))
len(maize_disease_dataset)

### Converting images to pixels

In [ ]:
x=[]
for image in maize_disease_dataset:
    img=cv2.imread(str(image))
    resized_img=cv2.resize(img,(180,180))
    x.append(resized_img)
x=np.array(x)
plt.imshow(x[0])
plt.axis('off')
plt.show()

In [ ]:
print(x[0])
print(x.dtype)

### Test, Train Split

In [ ]:
x_train,x_test=train_test_split(x,test_size=0.3,random_state=0) #Splitting
print(len(x_train))
print(len(x_test))

### Scalling the pixels to the range of 0...1

In [ ]:
# Scalling
x_train_scaled=x_train/255 #Scale of 0.,1.0
x_test_scaled=x_test/255
print(x_train_scaled[0])
plt.imshow(x_train_scaled[0]) #Display image
plt.axis('off')
plt.show()

### Creating augmentation function 1

In [ ]:
augmentation1=keras.Sequential([
    tf.keras.layers.RandomContrast(
    0.9, seed=None
)
])


### Augmenting the training data one

In [ ]:
augmented_training1=augmentation1(x_train_scaled)
print(augmented_training1[0])
current_dir=pathlib.Path(os.getcwd())
try:
    if not os.path.isdir(str(current_dir)+"\Augmented"):
        os.mkdir("Augmented")
    os.chdir(str(current_dir)+"\Augmented")
    for i in range(len(augmented_training1)):
        plt.imsave(f"Augmented1_img{i}.jpg",augmented_training1[i].numpy())
except:
    pass
finally:
    os.chdir(str(current_dir))
plt.imshow(augmented_training1[0])
plt.axis('off')
plt.show()

### Augmentation function 2

In [ ]:
augmentation2=Sequential([
    tf.keras.layers.RandomContrast(
    0.4, seed=None
    )
])


### Augmenting training data 2

In [ ]:
augmented_training2=augmentation2(x_train_scaled)
current_dir=pathlib.Path(os.getcwd())
try:
    if not os.path.isdir(str(current_dir)+"\Augmented"):
        os.mkdir("Augmented")
    os.chdir(str(current_dir)+"\Augmented")
    for i in range(len(augmented_training2)):
        plt.imsave(f"Augmented2_img{i}.jpg",augmented_training2[i].numpy())
except:
    pass
finally:
    os.chdir(str(current_dir))
plt.imshow(augmented_training2[0]) #Display our augmented image
plt.axis('off')
plt.show()

### Testing Image

In [ ]:
plt.imshow(x_test_scaled[1]) #Original test image,(Not augmented)
plt.axis('off')
plt.show()

### Augmenting test data 1

In [ ]:
augmented_testing1=augmentation1(x_test_scaled) #factor- 0.9
plt.imshow(augmented_testing1[1]) #Display augmented image of a particular index
plt.axis('off')
plt.show()

### Augmenting test data 2

In [ ]:
augmented_test2=augmentation2(x_test_scaled) #factor=0.4
plt.imshow(augmented_test2[1])
plt.axis('off')
plt.show()

# Classifying Original and Augmented images
### Loading the data

In [ ]:
maize_leaf_disease_dict={
    "Original":list(glob.glob(f"{data_dir}/data/*.jpg")),
    "Augmented":list(glob.glob(f"{os.getcwd()}/Augmented/*.jpg"))
}
print(len(maize_leaf_disease_dict["Original"]))
print(len(maize_leaf_disease_dict["Augmented"]))

In [ ]:
augmented_labels_dict={
    "Original":0,
    "Augmented":1
}

In [ ]:
print(len(maize_leaf_disease_dict["Augmented"]))

### Loading of the data

In [ ]:
x,y=[],[]
for augmented_original,images in maize_leaf_disease_dict.items():
    for image in images:
        img=cv2.imread(str(image))#Convert image to pixels
        resized_img=cv2.resize(img,(180,180))#Images to be of same size
        x.append(resized_img)#Pixels
        y.append(augmented_labels_dict[augmented_original])#Label Random 0,1
print(x[0])
print(y[:5])
x=np.array(x)
y=np.array(y)

In [ ]:
print(x.dtype)

In [ ]:
print(y)

### Test, Train split

In [ ]:
x_train,x_test,y_train, y_test=train_test_split(x,y,test_size=0.3,random_state=0) #Splitting
print(len(x_train))
print(len(x_test))
print(x_train.dtype)
print(x_train.dtype)

### Scaling

In [ ]:
x_train_scaled=x_train/255
x_test_scaled=x_test/255
print(x_train_scaled[0])

### Creating our CNN Model

In [ ]:
num_classes=2
model=Sequential([
    layers.Conv2D(16, 3, padding='same', activation='relu'),# Applied 16 filters of 3 X 3 Matrix
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    #128 neourons--> try and error
    layers.Dense(128,activation="relu"),
    layers.Dense(num_classes) #if 0 neuron is activated it means its Original, no activation(linear activation)
])

### Compiling the model

In [ ]:
model.compile(optimizer="adam",
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"]
             )

In [ ]:
print(x_train_scaled.dtype)
print(y_train.dtype)

### Training the CNN Model

In [ ]:
model.fit(x_train_scaled,y_train,epochs=10)

In [ ]:
model.evaluate(x_test_scaled,y_test)

In [ ]:
print(x_train_scaled[0])

In [ ]:
prediction=model.predict(x_train_scaled[])
score=tf.nn.softmax(prediction[0])
print(score)
print(np.argmax(score))

### Saving the model

In [ ]:
model.save("augmentation_model") 

### Reconstruct a new model from the final model

In [ ]:
reconstructed_model = keras.models.load_model("augmentation_model") 